# Simulate loan book
Douglas McLean, 28/6/2023

## The Ask
Simulate the loan book composition over time between five domiciled retail banks. The banks all start with zero loans written. They fund their loans with cash from their current accounts, savings book and by borrowing from each other and their central bank. Their savings accounts have different initial amounts but are drawn from a normal distribution of mean £1 billion and standard deviation £5 million (there is some £870bn in U.K. savings accounts spread amongst some 73 retail banks operating in the U.K.). Market interest rates fluctuate and are risky. The banks charge interest on their loans. They offer loans between £1000 and £25000. The banks change their interest rates competitively to maximise their returns. The  components needed to write a simulation model in Python over time are as follows.

### Key Modelling Points
1. Banks: Create a class to represent each bank, including attributes such as current account balance, savings account balance, loan book composition, interest rates, and methods to update these attributes based on simulation rules.

2. Loans: Generate loans with random amounts between £1000 and £25000. Assign these loans to the banks based on their funding sources and update the loan book composition of each bank accordingly.

3. Funding: Define the rules for banks to fund their loans. This can include using cash from their current accounts, savings accounts, borrowing from each other, and borrowing from the central bank. Implement methods to update the funding sources and loan book composition based on these rules.

4. Interest Rates: Define a model to simulate fluctuating and risky market interest rates. This can be based on historical data or a stochastic process. Update the interest rates of each bank competitively to maximize their returns. Consider the impact of interest rate changes on the loan book composition and profitability of each bank.

5. Simulation Loop: Write a loop to simulate the system over time. In each iteration, update the funding, interest rates, loan book composition, and profitability of each bank. Monitor and record the changes in the loan book composition and other relevant metrics at each time step.

6. Analysis and Visualization: After running the simulation, analyze and visualize the results to understand the dynamics of the loan book composition over time. This can involve plotting the loan book composition, profitability, and other key metrics for each bank.

## Potential Code Solution
Simulation model to study the loan book composition and dynamics between the five domiciled retail banks over time.



In [1]:
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import QuantLib as ql

In [2]:
class Bank:
    def __init__(self, name, current_balance, savings_balance, loan_book_composition, interest_rate):
        self.name = name
        self.current_balance = current_balance
        self.savings_balance = savings_balance
        self.loan_book_composition = loan_book_composition
        self.interest_rate = interest_rate

    def update_loan_book_composition(self, new_loan):
        self.loan_book_composition.append(new_loan)

    def update_funding_sources(self, loan_amount):
        # Update funding sources based on simulation rules
        # Example: Deduct loan amount from current balance
        self.current_balance -= loan_amount

    def update_interest_rate(self):
        # Update interest rate based on simulation rules
        # Example: Randomly fluctuating interest rate
        self.interest_rate = random.uniform(0.05, 0.1)

class Loan:
    def __init__(self, amount):
        self.amount = amount

def generate_loans(num_loans):
    loans = []
    for _ in range(num_loans):
        loan_amount = random.randint(1000, 25000)
        loans.append(Loan(loan_amount))
    return loans

def simulate_loan_book(num_banks, num_loans, num_iterations):
    banks = []
    for i in range(num_banks):
        bank = Bank(f"Bank {i+1}", 1000000, 500000, [], 0.05)
        banks.append(bank)

    for _ in range(num_iterations):
        loans = generate_loans(num_loans)
        for loan in loans:
            bank = random.choice(banks)
            bank.update_loan_book_composition(loan)
            bank.update_funding_sources(loan.amount)
            bank.update_interest_rate()

    # Perform analysis and visualization of loan book composition and other metrics
    # Example: Print loan book composition for each bank
    # for bank in banks:
    #    print(f"{bank.name}: {bank.loan_book_composition}")

# Run the simulation
num_banks = 5
num_loans = 100
num_iterations = 10

loan_book_object = simulate_loan_book(num_banks, num_loans, num_iterations)

Note: code provides basic framework for simulating the loan book composition. It needs customised and expanded further based on our specific requirements to incorporate customer default, loan amount and tenure tranches.

## Credit Default Modelling & Demographics
Now suppose that customer quality varies by probability of defaulting. Assume a range of default probabilities across a range of demographics. The banks’ loan books are then broadly stratified into tranches of credit default quality (high to low), loan amount (high to low amounts) and loan tenure (repayment length). Modify the Python code to accommodate these improvements

### Python modifications 
To accommodate customer quality variations by probability of defaulting, as well as stratify loan books into tranches of credit default quality, loan amount, and loan tenure, the following changes 1-6 are made:

1. Update the `Loan` class to include attributes for customer quality and probability of defaulting:


In [3]:
class Loan(Loan):
    def __init__(self, amount, tenure, customer_quality,):
        self.amount = amount
        self.tenure = tenure
        self.customer_quality = customer_quality
        self.interest_rate = self.calculate_interest_rate()
        self.default_probability = self.calculate_default_probability()
        self.defaulted = False

    def calculate_default_probability(self):
        # Calculate default probability based on customer quality
        # Example: Assign default probability based on predefined ranges
        if self.customer_quality == "high":
            return random.uniform(0.01, 0.05)
        elif self.customer_quality == "medium":
            return random.uniform(0.05, 0.15)
        elif self.customer_quality == "low":
            return random.uniform(0.15, 0.3)
        else:
            return 0.0
    def calculate_interest_rate(self):
        if self.customer_quality == "high":
            return round(random.uniform(0.05, 0.065), 4)
        elif self.customer_quality == "medium":
            return round(random.uniform(0.065, 0.08), 4)
        elif self.customer_quality == "low":
            return round(random.uniform(0.08, 0.1), 4)
        else:
            return 0.0


2. Modify the `generate_loans` function to generate loans with varying customer quality:

In [4]:
def generate_loans(num_loans):
    loans = []
    for _ in range(num_loans):
        loan_amount = random.randint(1000, 25000) #amounts in GBP Sterling
        loan_tenure = random.randint(12, 60) #months
        customer_quality = random.choice(["high", "medium", "low"])
        loans.append(Loan(loan_amount, loan_tenure, customer_quality))
    return loans

3. Update the `Bank` class to include methods for stratifying loan books based on credit default quality, loan amount, and loan tenure:

In [5]:
class Bank(Bank):
    # ...existing code...

    def stratify_loan_book_by_credit_quality(self, loans=None):
        if loans is None:
            loans = self.loan_book_composition
        # Stratify loan book by credit default quality
        low_quality_loans = [loan for loan in loans if loan.customer_quality == "low"]
        medium_quality_loans = [loan for loan in loans if loan.customer_quality == "medium"]
        high_quality_loans = [loan for loan in loans if loan.customer_quality == "high"]
        return high_quality_loans, medium_quality_loans, low_quality_loans

    def stratify_loan_book_by_loan_amount(self, loans=None):
        if loans is None:
            loans = self.loan_book_composition
        # Stratify loan book by loan amount
        low_amount_loans = [loan for loan in loans if loan.amount <= 5000]
        medium_amount_loans = [loan for loan in loans if 5000 < loan.amount <= 15000]
        high_amount_loans = [loan for loan in loans if loan.amount > 15000]
        # sorted_loans = sorted(self.loan_book_composition, key=lambda loan: loan.amount, reverse=True)
        return high_amount_loans, medium_amount_loans, low_amount_loans

    def stratify_loan_book_by_loan_tenure(self, loans=None):
        if loans is None:
            loans = self.loan_book_composition
        # Stratify loan book by loan tenure
        low_tenure_loans = [loan for loan in loans if loan.tenure <= 24]
        medium_tenure_loans = [loan for loan in loans if 24 < loan.tenure <= 48]
        high_tenure_loans = [loan for loan in loans if loan.tenure > 48]
        # sorted_loans = sorted(self.loan_book_composition, key=lambda loan: loan.tenure, reverse=True)
        return high_tenure_loans, medium_tenure_loans,low_tenure_loans


4. Modify the simulation loop in the `simulate_loan_book` function to include the loan stratification methods:

In [6]:
def simulate_loan_book(num_banks, num_loans, num_iterations):
    banks = []
    for i in range(num_banks):
        bank = Bank(f"Bank {i+1}", 1000000, 500000, [], 0.05)
        banks.append(bank)

    for _ in range(num_iterations):
        loans = generate_loans(num_loans)
        for loan in loans:
            bank = random.choice(banks)
            bank.update_loan_book_composition(loan)
            bank.update_funding_sources(loan.amount)
            bank.update_interest_rate()

We'll need to perform analysis and visualization of loan book composition and performance for each bank, considering the stratification methods:

5. Add a new method to the `Bank` class for analyzing the loan book composition and performance:

In [7]:
class Bank(Bank):
    # ...existing code...

    # Get 27 tranches
    def loan_tranches(self):
        loan_tranches_quality = list(self.stratify_loan_book_by_credit_quality())
        loan_tranches_quality_tenure = []
        for loan_tranche in loan_tranches_quality:
            loan_tranches_quality_tenure.extend(self.stratify_loan_book_by_loan_tenure(loan_tranche))
        loan_tranches_quality_tenure_amount = []
        for loan_tranche in loan_tranches_quality_tenure:
            loan_tranches_quality_tenure_amount.extend(self.stratify_loan_book_by_loan_amount(loan_tranche))
        return loan_tranches_quality_tenure_amount

    def analyze_loan_book(self, loans):
        # Analyze loan book composition and performance
        total_loans = len(loans)
        total_defaulted_loans = sum(1 for loan in loans if loan.defaulted)
        default_rate = total_defaulted_loans / total_loans

        # Add further analysis based on loan stratification if required

        return total_loans, total_defaulted_loans, default_rate

6. Update the `simulate_loan_book` function to analyze and visualize the loan book composition and performance for each bank:

In [8]:
def simulate_loan_book(num_banks, num_loans, num_iterations):
    banks = []
    for i in range(num_banks):
        bank = Bank(f"Bank {i+1}", 1000000000, 500000, [], 0.05)
        banks.append(bank)

    for _ in range(num_iterations):
        loans = generate_loans(num_loans)
        for loan in loans:
            bank = random.choice(banks)
            bank.update_loan_book_composition(loan)
            bank.update_funding_sources(loan.amount)
            bank.update_interest_rate()

    # for bank in banks:
    #     high_quality_loans, medium_quality_loans, low_quality_loans = bank.stratify_loan_book_by_credit_quality()
    #     sorted_loans_by_amount = bank.stratify_loan_book_by_loan_amount()
    #     sorted_loans_by_tenure = bank.stratify_loan_book_by_loan_tenure()

        # Analyze and visualize loan book composition and performance
        # total_loans, total_defaulted_loans, default_rate = bank.analyze_loan_book(bank.loan_book_composition)

        # Add further visualization or analysis based on loan stratification if required
    return banks

In [9]:
banks = simulate_loan_book(5, 10000, 38)

In [10]:
# HHH Tranche (Move this code to bank class to generate 27 tranches based on conditions like this)
# tenure Low 12 - 24, Medium 25-48, High 48-60
# amount Low 1000 - 5000, Medium 5001-15000, High 15001-25000
# credit quality Low, Medium, High
# loan_values = []
# for bank in banks:
# bank = banks[0]
# for loan in bank.loan_book_composition:
#     if loan.customer_quality == 'high' and loan.tenure > 48 and loan.amount > 15000:
#         tranche.append(loan)
    # loan_values.append({"amount": loan.amount, "tenure": loan.tenure, "quality": loan.customer_quality, "interest": loan.interest_rate})

# df = pd.DataFrame(loan_values)
# df = df[df['quality']=='high']
# df = df[df['tenure']>48]
# df = df[df['amount']>15000]
# df

In [11]:
loan_tranches = banks[0].loan_tranches()

In [12]:
len(loan_tranches)

27

By implementing these modifications, the Python code now accommodate customer quality variations by probability of defaulting and stratifies loan books into tranches based on credit default quality, loan amount, and loan tenure

## Visualisation
Visualize the tranches over time and write out to a CSV file based on simulation output, follow these steps:

In [13]:

# 1. Import the necessary libraries:
# import pandas as pd
# import matplotlib.pyplot as plt

# 2. After running your existing simulation code, you will have the `tranches` list, which contains the values of tranches over time.
# <add here>

# 3. Create a pandas DataFrame using the `tranches` list:
df = pd.DataFrame(tranches, columns=['Time Period', 'Tranche Value']) 
# Make sure to modify the column names accordingly if they differ from the example.

# 4. Visualize the tranches over time using a line plot:
plt.plot(df['Time Period'], df['Tranche Value'])
plt.xlabel('Time Period')
plt.ylabel('Tranche Value')
plt.title('Tranches over Time')
plt.xticks(rotation=45)
plt.show()

# 5. Save the visualization as an image:
plt.savefig('tranches_graph.png')

# 6. Write the tranches data to a CSV file:
df.to_csv('tranches_data.csv', index=False)

NameError: name 'tranches' is not defined

# End-to-End Code

The complete code for simulation and visualisation to look as follows:

```python
```




In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt

# Your existing code and simulation output here

# Create a pandas DataFrame from the tranches list df = pd.DataFrame(tranches, columns=['Time Period', 'Tranche Value'])

# Visualize the tranches over time
plt.plot(df['Time Period'], df['Tranche Value'])
plt.xlabel('Time Period') 
plt.ylabel('Tranche Value') 
plt.title('Tranches over Time')
plt.xticks(rotation=45)
plt.show()

# Save the graph as an image
plt.savefig('tranches_graph.png')

# Write the tranches data to a CSV file
df.to_csv('tranches_data.csv', index=False)

Make sure to adjust the column names and data accordingly based on your specific simulation output.

This code will generate a line graph of the tranches over time, save it as "tranches_graph.png", and write the data to a CSV file named "tranches_data.csv" without including the index column.